# Naive Bayes Classification



Intuitively, some classification problems seem succeptible to a strategy based on the likelihood of our sample being in a given class based on probability of the presense of the features. 

For example, the likelihood of an email with the phrase "Nigerian Prince" being spam is much higher than an email with the phrase "This is your monther. Why haven't you called?"

In terms of probabilities, we want to calculate $p(class | features)$ for each possible class and then choose the class with the max probability.

In order to estimate the probabilities we need to calculate $p(class | features)$, we can use a the technique of labeling some data, then calculating the probabilites.

###Bayes' Theorem

How to proceed? Given a labeled training set, it is straight forward to calculate $p(features|class)$. 

Flipping this over to get what we want looks like a job for Bayes' Theorem:

$$p(x)p(y|x) = p(y)p(x|y)$$

Use your algebra:

$$p(features)p(class|features) = p(class)p(features|class)$$

$$p(class|features) = \frac{p(class)p(features|class)}{p(features)}$$

### Chain Rule

This seems pretty managable, but there is one tiny hitch. There are potentially many dimensions to each feature vector, ${f_0, f_1, \ldots ,f_n}$. This means we have,

$$p(features|class) = p({f_0, f_1, \ldots ,f_n}|c)$$

Think about only 2 dimentions for a moment. If the features relate to the class independently of each other, we can rewrite:

$$p({f_0, f_1}|c) = p(f_1|c, f_0)p(f_0|c)$$

But since the probablility of $f_1$ is independent of $f_0$, we can repeat this operation through the fulll set of features to collapse to,

$$p({f_0, f_1}|c) = p(f_1|c)p(f_0|c)$$

So in general we can write:

$$p({f_0, f_1, \ldots, f_n}|c) = \Pi^n_i p(f_i|c)$$

### Naive Bayes Classification

Start with a representative labeled training set. Calculate $p(c_i)$ for each class. Calculate $p(f_j|c_i)$ for all feature dimensions. Classify new samples by finding the most likely class given the features:

$$p(class_i|{f_0, f_1, \ldots, f_n})= \frac{p(class_i)}{p({f_0, f_1, \ldots, f_n})} \Pi^n_j p(f_j|c_i)$$

## Problem 1: Classify even/odd binary numbers by the number of 0s and 1s.

* How many dimensions in the feature vector?
* Why might this work? 
* Why shouldn't it work?
* How well do you think this will work?

In [2]:
truth = ["odd", "even"]  # uses some human readiable text to make reading results easier
training_data = [ 
        ("{:05d}".format(
            int(str(bin(x)).replace("0b","")))
            , truth[int(x%2 == 0)]) 
                for x in range(1,32)]
training_data

[('00001', 'odd'),
 ('00010', 'even'),
 ('00011', 'odd'),
 ('00100', 'even'),
 ('00101', 'odd'),
 ('00110', 'even'),
 ('00111', 'odd'),
 ('01000', 'even'),
 ('01001', 'odd'),
 ('01010', 'even'),
 ('01011', 'odd'),
 ('01100', 'even'),
 ('01101', 'odd'),
 ('01110', 'even'),
 ('01111', 'odd'),
 ('10000', 'even'),
 ('10001', 'odd'),
 ('10010', 'even'),
 ('10011', 'odd'),
 ('10100', 'even'),
 ('10101', 'odd'),
 ('10110', 'even'),
 ('10111', 'odd'),
 ('11000', 'even'),
 ('11001', 'odd'),
 ('11010', 'even'),
 ('11011', 'odd'),
 ('11100', 'even'),
 ('11101', 'odd'),
 ('11110', 'even'),
 ('11111', 'odd')]

In [3]:
from operator import itemgetter

In [4]:
n_even = len([itemgetter(0)(x) for x in training_data if itemgetter(1)(x) == "even"])
n_odd = len([itemgetter(0)(x) for x in training_data if itemgetter(1)(x) == "odd"])
p_even = float(n_even)/len(training_data)
p_odd = float(n_odd)/len(training_data)
print "p(even) = {}  p(odd) = {}".format(p_even, p_odd)

p(even) = 0.483870967742  p(odd) = 0.516129032258


In [5]:
n_zeros = sum([itemgetter(0)(x).count("0") for x in training_data])
n_ones = sum([itemgetter(0)(x).count("1") for x in training_data])
total_characters = n_zeros + n_ones
p_zero = float(n_zeros)/total_characters
p_one = float(n_ones)/total_characters
print "total characters : {}   0s : {}  1s : {}".format(total_characters, p_zero, p_one)

total characters : 155   0s : 0.483870967742  1s : 0.516129032258


In [6]:
n_zeros_even = sum([itemgetter(0)(x).count("0") for x in training_data if itemgetter(1)(x) == "even"])
n_ones_even = sum([itemgetter(0)(x).count("1") for x in training_data if itemgetter(1)(x) == "even"])
p_zero_given_even = float(n_zeros_even)/(n_zeros_even + n_ones_even)
p_one_given_even = float(n_ones_even)/(n_zeros_even + n_ones_even)
print ("p(0|even) = {}   P(1|even) = {} ".format(p_zero_given_even,p_one_given_even))

p(0|even) = 0.573333333333   P(1|even) = 0.426666666667 


In [7]:
n_zeros_odd = sum([itemgetter(0)(x).count("0") for x in training_data if itemgetter(1)(x) == "odd"])
n_ones_odd = sum([itemgetter(0)(x).count("1") for x in training_data if itemgetter(1)(x) == "odd"])
p_zero_given_odd = float(n_zeros_odd)/(n_zeros_odd + n_ones_odd)
p_one_given_odd = 1.0 - p_zero_given_odd
print ("p(0|odd) = {}   P(1|odd) = {} ".format(p_zero_given_odd,p_one_given_odd))

p(0|odd) = 0.4   P(1|odd) = 0.6 


###Simplification  

$p({f_0, f_1, \ldots, f_n}) = const.$ for all calcuations, so we can omit it from the classification calculation.

In [8]:
def p_odd_given(sample):
    n_zeros = sample.count("0")
    n_ones = sample.count("1")
    return p_odd * (p_zero_given_odd**n_zeros) * (p_one_given_odd**n_ones)
  
def p_even_given(sample):
    n_zeros = sample.count("0")
    n_ones = sample.count("1")
    return p_even * (p_zero_given_even**n_zeros) * (p_one_given_even**n_ones)

In [9]:
correct = 0
for s, t in training_data:
    star = ""
    if truth[int(p_odd_given(s) < p_even_given(s))] == t:
        correct += 1
        star = "*"
    print "Pattern: {} --> p(odd) = {:1.3f} p(even) = {:1.3f} predict = {:4s} truth = {:4s} {}".format(
            s
            , p_odd_given(s)
            , p_even_given(s)
            , truth[int(p_odd_given(s) < p_even_given(s))]
            , t
            , star)


Pattern: 00001 --> p(odd) = 0.008 p(even) = 0.022 predict = even truth = odd  
Pattern: 00010 --> p(odd) = 0.008 p(even) = 0.022 predict = even truth = even *
Pattern: 00011 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = odd  
Pattern: 00100 --> p(odd) = 0.008 p(even) = 0.022 predict = even truth = even *
Pattern: 00101 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = odd  
Pattern: 00110 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = even *
Pattern: 00111 --> p(odd) = 0.018 p(even) = 0.012 predict = odd  truth = odd  *
Pattern: 01000 --> p(odd) = 0.008 p(even) = 0.022 predict = even truth = even *
Pattern: 01001 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = odd  
Pattern: 01010 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = even *
Pattern: 01011 --> p(odd) = 0.018 p(even) = 0.012 predict = odd  truth = odd  *
Pattern: 01100 --> p(odd) = 0.012 p(even) = 0.017 predict = even truth = even *
Pattern: 01101 --> p(odd) = 0.018 p(even) = 

In [10]:
print "="*50
baseline_correct = sum([1 for x in training_data if itemgetter(1)(x) == "odd"]) # if we guessed all odd
print "Baseline (guess all \"odd\")"
print "Total patterns: {}  Total correct: {} ({:2.2%})".format(
        len(training_data)
        , baseline_correct
        , float(baseline_correct)/len(training_data))
print "="*50
print "Naive Bayes Classifier"
print "Total patterns: {}  Total correct: {} ({:2.2%})".format(
        len(training_data)
        , correct
        , float(correct)/len(training_data))
print "="*50

Baseline (guess all "odd")
Total patterns: 31  Total correct: 16 (51.61%)
Naive Bayes Classifier
Total patterns: 31  Total correct: 21 (67.74%)


###Thoughts: 

* Comparing to best naive baseline might not be the same as comparing to random.
* Comparing to best naive baseline might reveal pathologies of your measurement/cost assumptions or poorly balanced training set.
* Is it crazy to not use ideas of mechanism (i.e. do a 1 bit check on the LST of the binary number!)?
* How should we think about choosing features for machine learning when we have only tenuous ideas of mechanism?

##Problem 2: Classify Tweets by Topics

Make two sets of tweets with simple filters, don't work too much about what is in them.

    ./search_api.py -f"lang:en (dog OR cat)" json | gnacs.py | cut -d"|" -f3  | sort -u | head -qn85 > pets.txt
    ./search_api.py -f"lang:en (hat OR coat)" json | gnacs.py | cut -d"|" -f3  | sort -u | head -qn85 > hats.txt

In [11]:
corpus = open("pets.txt","rb").readlines()
n_pets = len(corpus)
corpus.extend(open("hats.txt","rb").readlines())
n_hats = len(corpus) - n_pets
Y = [0]*n_pets + [1]*n_hats
print "corpus size = {}   n pets = {}   n hats = {}".format(len(corpus), n_pets, n_hats)

corpus size = 170   n pets = 85   n hats = 85


In [12]:
print corpus[2]
print corpus[12]
print corpus[20]
print corpus[130]
print corpus[150]

@ATLTurnUpRadio can I send u some music big dog

**DARLING KITTEN DUO OF DELIGHT NEEDS A LOVING CAT PARENT - PLEASE GRANT PANDA &amp; LAYLA A DEATH ROW PARDON!!!***... http://t.co/RhriqIFz1Q

How chubby is this cat btw! Hahaha http://t.co/fwziGFOlce via

RT @hat_films: I liked a @YouTube video http://t.co/wyVzJByRLd BAFTA Young Game Designers: Winners Ceremony

RT @XFilesbutemoji: đ đŚscully guess what!  đŠyou finally got your high school diploma đ đŚno, i found this funny hat đ đŚi'm a wizard!



### Scikit Learn

Like with all of the other other fundamental algorithm RSTs, don't use the code indended to demonstrate how the algorithm works. Instead, use Scikit Learn.  

In this case, we take a big shortcut by using the term frequency inverse document frequency (tfidf) vectorizer. Recall, this weights each term by the number of appeances, but decreases the weights as it appears in too many documents to be informative.  Scikit Learn uses sparse matricies for space efficiency.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(min_df=2)
X = vec.fit_transform(corpus)

In [18]:
print corpus[101]
print X[101]


Im gonna eat my hat

  (0, 141)	0.331415548236
  (0, 98)	0.642536490438
  (0, 53)	0.642536490438
  (0, 84)	0.253884720489


In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, Y)
print(clf.predict(X[12]))
print(clf.predict(X[22]))
print(clf.predict(X[32]))
print(clf.predict(X[120]))
print(clf.predict(X[130]))
print(clf.predict(X[140]))

[0]
[0]
[0]
[1]
[1]
[1]


In [16]:
correct = 0
for i, doc in enumerate(corpus):
    if Y[i] == clf.predict(X[i]):
        correct += 1
    else:
        print i, Y[i], clf.predict(X[i]), doc
print "correct = {} ({:%})".format(correct, float(correct)/len(corpus))

correct = 170 (100.000000%)


##How does it generalize?

* Where does the opportunity for this model to generalize come from? Compare this to bias-variance tradeoff in Logistic Regression.
* How well do you think this model generalizes?

In [17]:
print "PETS"
print clf.predict(vec.transform(["My dog has fleas"]))  # seems clear
print clf.predict(vec.transform(["Grab your cat and moat"])) # also clear
print clf.predict(vec.transform(["This pet is a hamster"])) # not in the original word list, but pet nonetheless
print clf.predict(vec.transform(["socks can be the name of a pet"])) # pet
print clf.predict(vec.transform(["But hats is not usually the name of a pet"])) # pet
print "OUTERWARE"
print clf.predict(vec.transform(["Grab your hat and coat"])) # obvious
print clf.predict(vec.transform(["My socks and shoes are dirty"])) # obvious but too far?
print clf.predict(vec.transform(["These come in hard, tinfoil, thinking and mortar boards"])) # tricky
print clf.predict(vec.transform(["Redhat's logo is a fedora"])) # ok
print clf.predict(vec.transform(["The cat in the hat"])) # mixed, confound it!
print clf.predict(vec.transform(["Hiding a haircut under there?"])) # infered?
print clf.predict(vec.transform(["Mackinaw or Toque?"])) # Too far?

PETS
[0]
[0]
[0]
[1]
[1]
OUTERWARE
[1]
[1]
[1]
[1]
[1]
[0]
[0]
